# Analisis de señales raw 
### ZEMG V1.0

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pandas as pd



def cargar_datos_desde_archivos(archivos):
    data_frames = []

    for archivo in archivos:
        # Leer el archivo CSV y almacenar los datos en un DataFrame, omitiendo la primera fila
        df = pd.read_csv(archivo, skiprows=1, names=["Muestras", "Valores"])

        # Extraer el nombre del dedo de la ruta del archivo
        dedo_nombre = archivo.split("/")[1].split(".")[0]

        global ventana
        ventana=5
        df["MovMean"]=df["Valores"].rolling(ventana).mean()

        # Agregar el nombre del dedo como una columna
        df["Dedo"] = dedo_nombre

        data_frames.append(df)

    # Concatenar todos los DataFrames en uno solo
    data = pd.concat(data_frames, ignore_index=True)

    global datos_cargados
    datos_cargados = data


def graficar_datos():
    if datos_cargados is None:
        raise ValueError(
            "Los datos no han sido cargados. Llama a 'cargar_datos_desde_archivos' primero."
        )

    # Crear un gráfico de líneas interactivo utilizando Plotly Express
    fig = px.line(
        datos_cargados,
        template="plotly_dark",
        x="Muestras",
        y="Valores",
        color="Dedo",
        title="Gráfico de Varios Dedos",
    )
   

    # Mostrar el gráfico interactivo utilizando Plotly
    fig.show()

def graficar_datos_movMean():
    if datos_cargados is None:
        raise ValueError(
            "Los datos no han sido cargados. Llama a 'cargar_datos_desde_archivos' primero."
        )

    # Crear un gráfico de líneas interactivo utilizando Plotly Express
    fig = px.line(
        datos_cargados,
        template="plotly_dark",
        x="Muestras",
        y="MovMean",
        color="Dedo",
        title="Gráfico de Varios Dedos - Movmean = " + str(ventana),
    )
    

    # Mostrar el gráfico interactivo utilizando Plotly
    fig.show()


# Lista de nombres de archivos CSV que deseas graficar juntos
archivos_a_graficar = [
    'Records/Indice.csv', 'Records/Medio.csv', 'Records/Anular.csv', 'Records/Meñique.csv', 'Records/Pulgar.csv','Records/Puño.csv','Records/Puño Flexionado.csv'
]

# Cargar los datos desde los archivos una vez
cargar_datos_desde_archivos(archivos_a_graficar)



## Graficos de la señal cruda de cada dedo y de la movmean

In [4]:
graficar_datos()
graficar_datos_movMean()

## Analisis frecuencial (FFT)

In [5]:
# Selecciona el rango de muestras de interés (440 a 500) del dedo "Anular"
muestras_rango_anular = datos_cargados[
    (datos_cargados['Muestras'] >= 100) & 
    (datos_cargados['Muestras'] <= 1400) & 
    (datos_cargados['Dedo'] == 'Anular')
]['Valores']

# Obtén los valores de la columna 'Valores' en ese rango de muestras del dedo "Anular"


# Realiza la FFT de los valores en el rango seleccionado del dedo "Anular"
fft_valores_rango_anular = np.fft.fft(muestras_rango_anular)

# Encuentra la magnitud de las amplitudes de la FFT del dedo "Anular"
magnitud_fft_anular = np.abs(fft_valores_rango_anular)

# Encuentra las frecuencias correspondientes para el dedo "Anular"
frecuencias_anular = np.fft.fftfreq(len(muestras_rango_anular))

mitad_frecuencias_anular = frecuencias_anular[:len(frecuencias_anular)//2]
mitad_magnitudes_fft_anular = np.abs(fft_valores_rango_anular)[:len(magnitud_fft_anular)//2]

# Selecciona el rango de muestras de interés (450 a 550) del dedo "Índice"
muestras_rango_indice = datos_cargados[
    (datos_cargados['Muestras'] >= 100) & 
    (datos_cargados['Muestras'] <= 1400) & 
    (datos_cargados['Dedo'] == 'Indice')
]

# Obtén los valores de la columna 'Valores' en ese rango de muestras del dedo "Índice"
valores_rango_indice = muestras_rango_indice['Valores']

# Realiza la FFT de los valores en el rango seleccionado del dedo "Índice"
fft_valores_rango_indice = np.fft.fft(valores_rango_indice)

# Encuentra la magnitud de las amplitudes de la FFT del dedo "Índice"
magnitud_fft_indice = np.abs(fft_valores_rango_indice)

# Encuentra las frecuencias correspondientes para el dedo "Índice"
frecuencias_indice = np.fft.fftfreq(len(valores_rango_indice))
mitad_frecuencias_indice = frecuencias_indice[:len(frecuencias_indice)//2]
mitad_magnitudes_fft_indice = np.abs(fft_valores_rango_indice)[:len(magnitud_fft_indice)//2]



# Crear el gráfico de la FFT del dedo "Índice" con Plotly
fig_fft_indice = px.line(
    x=mitad_frecuencias_indice,
    y=mitad_magnitudes_fft_indice,
    template="plotly_dark",
    title="FFT del Dedo Índice",
)


# Crear el gráfico de la FFT del dedo "Anular" con Plotly
fig_fft_anular = px.line(
    x=mitad_frecuencias_anular,
    y=mitad_magnitudes_fft_anular,
    template="plotly_dark",
    title="FFT del Dedo Anular",
)


# Mostrar los gráficos interactivos utilizando Plotly
fig_fft_indice.show()
fig_fft_anular.show()


## Transformada Wavelet

In [6]:
import pywt
import plotly.graph_objs as go
from plotly.subplots import make_subplots


muestras_rango_anular = datos_cargados[
    (datos_cargados['Muestras'] >= 100) & 
    (datos_cargados['Muestras'] <= 1000) & 
    (datos_cargados['Dedo'] == 'Anular')
]['Valores']

muestras_rango_indice = datos_cargados[
    (datos_cargados['Muestras'] >= 100) & 
    (datos_cargados['Muestras'] <= 1400) & 
    (datos_cargados['Dedo'] == 'Indice')
]['Valores']

Movmean_anular = datos_cargados[
    (datos_cargados['Muestras'] >= 100) & 
    (datos_cargados['Muestras'] <= 1000) & 
    (datos_cargados['Dedo'] == 'Anular')
]['MovMean']

basetime=list(range(100, 1001));



# Calcular la descomposición de la señal usando wavedec (transformada wavelet discreta)
wavelet = 'db1'  
nivel_descomposicion = 5  
coefs_anular = pywt.wavedec(muestras_rango_anular, wavelet, level=nivel_descomposicion)
coefs_indice = pywt.wavedec(muestras_rango_indice, wavelet, level=nivel_descomposicion)

# Eliminar los coeficientes 5 y 6 (índices 4 y 5) de cada lista de coeficientes
coefs_anular_reconstruccion = coefs_anular[:4] + [None] * 2
coefs_indice_reconstruccion = coefs_indice[:4] + [None] * 2

# Reconstrucción de la señal sin los coeficientes 5 y 6
senal_reconstruida_anular = pywt.waverec(coefs_anular_reconstruccion, wavelet)
senal_reconstruida_indice = pywt.waverec(coefs_indice_reconstruccion, wavelet)

mav_indice = np.mean(np.abs(muestras_rango_indice))
mav_anular = np.mean(np.abs(muestras_rango_anular))



# print(f"El Valor Medio Absoluto (MAV) del indice es: {mav_indice}")
# print(f"El Valor Medio Absoluto (MAV) del anular es: {mav_anular}")


# Gráficos de cada coeficientes wavelet
fig = make_subplots(rows=len(coefs_anular), cols=1, subplot_titles=[f'Coeficiente {i+1}' for i in range(len(coefs_anular))])
for i, (coef_anular, coef_indice) in enumerate(zip(coefs_anular, coefs_indice), start=1):
    fig.add_trace(go.Scatter(y=coef_anular, mode='lines', name='Anular Coef' + str(i)), row=i, col=1)
    fig.add_trace(go.Scatter(y=coef_indice, mode='lines', name='Índice Coef' + str(i)), row=i, col=1)
    fig.update_xaxes(title_text='Muestras', row=i, col=1)
    fig.update_yaxes(title_text='Coeficiente', row=i, col=1)

fig.update_layout(height=1200, width=1600, title_text="Coeficientes",template="plotly_dark")
fig.show()


# Gráficos de la reconstrucción de la wavelet sin los coeficientes 5 y 6 - Comparando con la señal original y la filtrada (MovMean)

fig = px.line(template="plotly_dark",title="Reconstrucción Wavelet vs Señal Original vs MovMean")
fig.add_scatter(x=basetime,y=muestras_rango_anular, mode='lines', name='Muestras Anular')
fig.add_scatter(x=basetime,y=senal_reconstruida_anular, mode='lines', name='Señal Anular Reconstruida')
fig.add_scatter(x=basetime, y=Movmean_anular, mode='lines', name='MovMean')
fig.update_xaxes(title_text='Muestras')
fig.update_yaxes(title_text='Valores')

fig.show()